## BUDGET run:

run dir: SS-run-sets/SalishSea/nemo3.6/tereza/FUTURE_runs/BUDGET_2017_RUN
branch of run dir: TJSJ_future_runs
config dir: NEMO-3.6-code/NEMOGCM/CONFIG/SalishSeaCast
branch of config dir: TJSJ_future_forprod
results on graham: /scratch/tjarnik/BUDGET
results on salish: /data/tjarniko/results/FUTURE_runs/BUDGET/01jan17_RIVTA2/

Note: 01jan17_RIVTA4 will have E3TSNAP which is midnight instantaneous tracer content!

Equivalence notebook: 

http://localhost:8802/notebooks/notebooks/RIVER_PAPER/BUDGET/equivalence_check-RIVTA_and_PRDREM_METRICS.ipynb
    
   
    

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [37]:
import numpy as np
# import xarray as xr
import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# from mpl_toolkits.basemap import Basemap
# from scipy.io import loadmat
from cmocean import cm
import pandas as pd
import warnings
import pickle
import arrow
#from cartopy import crs, feature
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from salishsea_tools import viz_tools
import netCDF4 as nc
import glob
from decimal import Decimal

#import LambertConformalTicks as lct

%matplotlib inline
plt.rcParams['font.size'] = 11
warnings.simplefilter('ignore')

, diatoms, dic (expected to go away after 60 days)

## Part 2: Start budgeting!!

### Part 2: What sort of metrics are coming out, do they make sense, do I understand the units, and what does the budget look like? - notes in http://localhost:8802/notebooks/notebooks/RIVER_PAPER/BUDGET/equivalence_check-RIVTA_and_PRDREM_METRICS.ipynb

Governing equations of the system (as written in MM16):

$\frac{\partial DIC}{\partial t } = - (U_{N} + U_{PC} - Rm_{N}) * R_{C:N} + {physical transport} + {airsea flux} $ 

$\frac{\partial TA}{\partial t } = U_{NO3-} - 2Rm_{NO3-} - U_{NH4+} + Rm_{NH4+} + (U_{PO4} + Rm_{PO4})* R_{N:P} + {physical transport}$ 


decision i've made: 

everything calculated 20 grid cells (10 km) in from boundary


- things we need: 

- carbon content of domain daily (mmol)
            - one way to get that is (daily average DIC) * (e1t*e2t*e3t)
            - one way to get that is through DIC_e3tsnap * (e1t*e2t)
                -instantaneous-at-midnight DIC concentration * e3t at that time (changes due to tidal range)
- change in daily carbon content (subtract daily values of the above)

this change should be equal to the sum of:

- river carbon input (RIV_DIC (mmol/s))
- carbon taken up by phytoplankton (mmol/s)
- carbon remineralized (mmol/s)
- carbon going through the western boundary (mmol/s)
- carbon going through the northern boundary (mmol/s)
- carbon going through the air-sea boundary (air_sea_carbon * e1t*e2t, mmol/s)

            

In [62]:
##welll how different are these runs anyway

q4 = '/data/tjarniko/results/FUTURE_runs/BUDGET/01jan17_RIVTA4/'
q2 = '/data/tjarniko/results/FUTURE_runs/BUDGET/01jan17_RIVTA2/'
#q = '/data/tjarniko/results/FUTURE_runs/BUDGET/'
#this is in mmol N /s

#prdrem_02jan = nc.Dataset(f'{q}SalishSeaCast_1d_20170101_20170130_PRDREM_T_20170102-20170102.nc')
prdrem_02jan4 = nc.Dataset(f'{q4}SalishSeaCast_1d_20170101_20170130_PRDREM_T_20170117-20170117.nc')
carp_02jan4 = nc.Dataset(f'{q4}SalishSeaCast_1d_20170101_20170130_carp_T_20170117-20170117.nc')
dianu_02jan4 = nc.Dataset(f'{q4}SalishSeaCast_1d_20170101_20170130_dian_U_20170117-20170117.nc')
dianv_02jan4 = nc.Dataset(f'{q4}SalishSeaCast_1d_20170101_20170130_dian_V_20170117-20170117.nc')
# prdrem_02jan2 = nc.Dataset(f'{q2}SalishSeaCast_1d_20170101_20170130_PRDREM_T_20170117-20170117.nc')
# carp_02jan2 = nc.Dataset(f'{q2}SalishSeaCast_1d_20170101_20170130_carp_T_20170117-20170117.nc')
# print(prdrem_02jan)
# print(carp_02jan)

In [53]:
# dimensional conversion variables 

#meshmask
mmk = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
e2t = (mmk['e2t'][0,:,:])
e1t = (mmk['e1t'][0,:,:])
surfar_m2 = e1t * e2t
tmask_orig = mmk['tmask'][0,:,:,:]
tmask = (mmk['tmask'][0,:,:,:])

# cut edges
tmask[:,878:898,:] = 0
tmask[:,:,0:20] = 0

surfar_m2[878:898,:] = 0
surfar_m2[:,0:20] = 0

surfar_m2_broad = np.zeros([40,898,398])

for i in range(0,40):
    surfar_m2_broad[i,:,:] = surfar_m2

surfar_m2_mask = surfar_m2_broad * tmask

#seconds in day
sid = 60*60*24

In [61]:
### GET OUT THE NUMBERS!

# prdrem_02jan4
# carp_02jan4

## TOTAL DIC IN SYSTEM IN MMOLS
e3t = carp_02jan4['e3t'][0,:,:,:]
DIC = carp_02jan4['dissolved_inorganic_carbon'][0,:,:,:]

DIC_mmol = DIC * e3t* surfar_m2_mask 
DIC_mmol_tot = np.nansum(DIC_mmol)
print('mmols DIC in system:')
print(f"{Decimal(DIC_mmol_tot):.8E}")

#### total DIC input in rivers
# ?? when multiply river_DIC by tmask, stuff gets cut out? tthere must be a river coming in near the boundary 

riverine_DIC = prdrem_02jan4['riverine DIC'][0,:,:,:]  * tmask
##mmols/Day
riv_DIC_tot = np.nansum(riverine_DIC) * sid

print('DIC coming in through rivers (mmol/day):')
print(f"{Decimal(riv_DIC_tot):.8E}")

PRD_DIC = prdrem_02jan4['PRD DIC'][0,:,:,:]  * tmask
PRD_DIC_tot = np.nansum(PRD_DIC) * sid 
print('DIC taken up by phyto (mmol/day):')
print(f"{Decimal(PRD_DIC_tot):.8E}")

REM_DIC = prdrem_02jan4['REM DIC'][0,:,:,:]  * tmask
REM_DIC_tot = np.nansum(REM_DIC) * sid 
print('DIC remineralized! (mmol/day):')
print(f"{Decimal(REM_DIC_tot):.8E}")


DIC_UT = dianu_02jan4['DIC_UT'][0,:,300:500,20]

# river carbon input (RIV_DIC (mmol/s))
# carbon taken up by phytoplankton (mmol/s)
# carbon remineralized (mmol/s)
# carbon going through the western boundary (mmol/s)
# carbon going through the northern boundary (mmol/s)
# carbon going through the air-sea boundary (air_sea_carbon * e1t*e2t, mmol/s)

mmols DIC in system:
4.76602603E+15
DIC coming in through rivers (mmol/day):
3.77569339E+11
DIC taken up by phyto (mmol/day):
-7.73919954E+10
DIC remineralized! (mmol/day):
7.68888612E+10


In [63]:
dianu_02jan4

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: SalishSeaCast_1d_20170101_20170130_dian_U
    description: Chem transports
    title: Chem transports
    Conventions: CF-1.6
    timeStamp: 2021-Jul-31 11:39:45 GMT
    uuid: 8942047a-9d8d-4f16-a89f-8fe32b4e65b5
    dimensions(sizes): axis_nbounds(2), x(398), y(898), nvertex(4), depthu(40), time_counter(1)
    variables(dimensions): float32 nav_lat(y,x), float32 nav_lon(y,x), float32 bounds_nav_lon(y,x,nvertex), float32 bounds_nav_lat(y,x,nvertex), float32 area(y,x), float32 depthu(depth), float32 depthu_bounds(depth,axis_nbounds), float64 time_centered(time_counter), float64 time_centered_bounds(time_counter,axis_nbounds), float64 time_counter(time_counter), float64 time_counter_bounds(time_counter,axis_nbounds), float32 DIC_UT(time_counter,depth,y,x), float32 TA_UT(time_counter,depth,y,x)
    groups: 

In [6]:
REDRAT = 106/16

PRD_DIC = prdrem_02jan['PRD DIC'][:]
UNC_DIC = prdrem_02jan['Uncoupled DIC uptake'][:]
PRD_NIT = prdrem_02jan['PRD nitrate'][:]
PRD_NH = prdrem_02jan['PRD ammonium'][:]
PRD_DIC_check = (PRD_NIT+PRD_NH-UNC_DIC)*REDRAT 


#remineralization metrics
REM_PON = prdrem_02jan['REM particulate_organic_nitrogen'][:]

#print(REM_PON[0,:,250,250])
REM_DON = prdrem_02jan['REM dissolved_organic_nitrogen'][:]

REM_DIC_check = -1 * (REM_PON+ REM_DON)*REDRAT

REM_DIC = prdrem_02jan['REM DIC'][:]
REM_TA = prdrem_02jan['REM TA'][:]
print('rem dic')
print(REM_DIC[0,:,250,250])
print('rem dic check (from REM_PON and REM_DON)')
print(REM_DIC_check[0,:,250,250])
print('rem dic - rem dic check (from REM_PON and REM_DON)')
print(REM_DIC[0,:,250,250]-REM_DIC_check[0,:,250,250])
# print(REM_TA[0,:,250,250])
print()
print('prd dic')
print(PRD_DIC[0,:,250,250])
print('prd dic check (from nitrate, - uncoupled carbon uptake)')
print(PRD_DIC_check[0,:,250,250])
#uptake DIC so that it's not confusing
print('prd dic - (prd dic check)')
print(PRD_DIC[0,:,250,250] - PRD_DIC_check[0,:,250,250] )
print('at x=250, y = 250, calculated. remineralization of DIC, mmol/S')
#print(REM_DIC[0,:,250,250])
#print(REM_DIC_check[0,:,250,250])
#print(REM_DIC[0,:,250,250]-REM_DIC_check[0,:,250,250])

# print('at x=250, y = 250, calc. uptake of DIC, mmol/S')
# print(UPT_DIC[0,:,250,250])

# print('at x=250, y = 250, sum of calcul-remin and calcul-uptake \n doesnt include uncoupled C uptake')
# print(REM_DIC[0,:,250,250]+UPT_DIC[0,:,250,250])

# print('at x=250, y = 250, output of PRD DIC (aka DEL DIC, should be the sameish)')
# print(DEL_DIC[0,:,250,250])

# rem_nit = prdrem_02jan['REM nitrate'][:]
# prd_nit = prdrem_02jan['PRD nitrate'][:]
# prd_nh4 = prdrem_02jan['PRD ammonium'][:]

# prd_DIC = prdrem_02jan['PRD DIC'][:]
# rem_DIC = prdrem_02jan['REM DIC'][:]
# #REM ammonium
# sum_nit = prd_nit[0,:,250,250] + prd_nh4[0,:,250,250]
# # print(prd_nit[0,:,250,250])
# # print(prd_nh4[0,:,250,250])
# print('sum nitrogen uptake (at one point, x = 250, y = 250, depth profile)')
# print(sum_nit)
# print('nitrogen uptake x redfield ratio')
# print(sum_nit * 106/16)
# print()
# print('DIC uptake (PRD_DIC)')
# print(prd_DIC[0,:,250,250])""
# print('DIC remineralization (REM_DIC)')
# print(rem_DIC[0,:,250,250])

print()

rem dic
[0.13061611354351044 0.13079750537872314 0.13094507157802582
 0.13107053935527802 0.13118669390678406 0.13130271434783936
 0.13143055140972137 0.13165511190891266 0.13193607330322266
 0.13218669593334198 0.13247905671596527 0.13292960822582245
 0.1337435096502304 0.13528099656105042 0.1382402628660202
 0.14397591352462769 0.15507832169532776 0.17649583518505096
 0.21743907034397125 0.29425081610679626 0.4336138367652893
 0.6705707311630249 1.0288505554199219 1.4428406953811646
 1.787988305091858 1.9713507890701294 2.043449640274048 1.2755796909332275
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
rem dic check (from REM_PON and REM_DON)
[0.13061611354351044 0.13079750537872314 0.130945086479187
 0.13107052445411682 0.13118669390678406 0.13130269944667816
 0.13143055140972137 0.13165511190891266 0.13193607330322266
 0.13218671083450317 0.13247907161712646 0.13292960822582245
 0.1337435096502304 0.13528099656105042 0.1382402628660202
 0.14397592842578888 0.15507832169532776 0.

In [7]:
## are there any rivers that come in within 20 grid cells of the boundary?


RIV_DIC = prdrem_02jan['riverine DIC'][:]

print(np.shape(RIV_DIC))

print(np.nansum(RIV_DIC[0,0,:,:]))

(1, 40, 898, 398)
1711347.2
